In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

In [2]:
today = date.today()
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-08-22
yesterday: 2025-08-21


In [3]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [4]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Tables in the process

In [6]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [7]:
def show_buy(const, name):
    sql = """
    SELECT * 
    FROM buy 
    WHERE name = '{}'
    """.format(name)
    
    df_buy = pd.read_sql(sql, const)
    df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
    df_buy.rename(columns={'volbuy':'shares'},inplace=True)
    df_buy['shares'] = df_buy['shares'].astype('int64')
    return df_buy.style.format(format_dict)

In [8]:
def show_dividend(const, name):
    sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
    """.format(name)
    
    df_dividend = pd.read_sql(sql, const)
    df_dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
    df_dividend.columns = df_dividend.columns.str.lower()
    df_dividend['shares'] = df_dividend['shares'].astype('int64')
    df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
    df_dividend['paiddate'] = pd.to_datetime(df_dividend['paiddate'])
    return df_dividend.style.format(format_dict)

In [9]:
def show_stocks(conlite, name):
    sql = """
    SELECT * 
    FROM stocks 
    WHERE name = '{}'
    """.format(name)
    
    df_stocks = pd.read_sql(sql, conlite)
    return df_stocks.style.format(format_dict)

In [10]:
def update_buy(const, name, new_qty, new_price):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [11]:
def update_dividend(const, name, new_qty):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [12]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'new_unit_cost': new_unit_cost, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {result.rowcount}"

### End of Definition

## Begin of  Buy transaction  

In [ ]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
df_buys_latest = pd.read_sql(sql, conpf)
df_buys_latest.style.format(format_dict)

In [ ]:
transaction = 'B'

names = df_buys_latest['name']
name = names.to_string(index=False)

sr_qty = df_buys_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = df_buys_latest['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

print(transaction, name, qty, buy_price)

### Begin of Update buy table in MySQL database from buy transaction

In [ ]:
sql = """
    SELECT *
    FROM buy 
    WHERE name = '{}'
""".format(name)
    
df_buy = pd.read_sql(sql, const)
df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
df_buy.rename(columns={'volbuy':'shares'},inplace=True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy.style.format(format_dict)

In [ ]:
new_qty = int(df_buy['shares'].iloc[0] + qty)
new_qty

In [ ]:
new_unit_cost = ((df_buy['price'].iloc[0] * df_buy['shares'].iloc[0]) + (qty * buy_price)) / new_qty
new_unit_cost 

In [ ]:
print(transaction, name, new_qty, new_unit_cost)

### In case there is the same stock name in port

In [ ]:
update_buy(const, name, new_qty, new_unit_cost)

In [ ]:
show_buy(const, name)

### In case of totally new stock

In [ ]:
if df_buy.empty:
    rcd = {
        'name': 'CPF',
        'date': '2025-08-21',
        'volbuy': 4000,
        'price': 24.1,
        'volsell': 0,
        'volbal': 0,
        'active': 1,
        'dividend': 0,
        'period': 4,
        'grade': 'A1'
    }
    sqlIns = text("""
    INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
    VALUES (:name, :date, :volbuy, :price, :volsell, :volbal, :active, :dividend, :period, :grade)
    """)
    result = const.execute(sqlIns, rcd)
    result.rowcount

In [ ]:
show_buy(const, name)

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [ ]:
show_dividend(const, name)

In [ ]:
sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
""".format(name)
    
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['shares'] = df_dividend['shares'].astype('int64')

df_dividend.style.format(format_dict)

In [ ]:
new_qty = df_dividend['shares'].iloc[0] + qty
new_qty

In [ ]:
print(name, new_qty)

In [ ]:
update_dividend(const, name, new_qty)

In [ ]:
show_dividend(const, name)

### In case of totally new dividend

In [ ]:
if df_dividend.empty:
    rcd = {
        'name': 'CPF',
        'q4': '0',
        'q3': '0',
        'q2': '1.00',
        'q1': '0',    
        'dividend': '1.00',      
        'shares': 4000,
        'xdate': '2025-08-29',
        'paiddate': '2025-09-12',
        'kind': ' ',
        'actual': 1
    }
    sqlIns = text("""
    INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
    VALUES (:name, :q4, :q3, :q2, :q1, :dividend, :shares, :xdate, :paiddate, :kind, :actual)
    """)
    print(sqlIns)

In [ ]:
if df_dividend.empty:
    result = const.execute(sqlIns, rcd)
    result.rowcount

In [ ]:
show_dividend(const, name)

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [ ]:
sql = """
    SELECT name, cost, available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)
print(sql)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks

In [ ]:
new_qty = df_stocks['available_qty'].iloc[0] + qty
new_qty

In [ ]:
new_unit_cost = ((df_stocks['cost'].iloc[0] * df_stocks['available_qty'].iloc[0]) + (qty * buy_price)) / new_qty
new_unit_cost 

In [ ]:
new_buy_target = 24.1
new_sell_target = 24.30
new_buy_qty = 5_000
print(transaction, name, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty) 

In [ ]:
update_stock(conlite, name, qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

In [ ]:
show_stocks(conlite, name)

### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy transaction

## Begin of Sale transaction

In [ ]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
df_sells_latest = pd.read_sql(sql, conpf)
df_sells_latest.style.format(format_dict)

### In case of sale, retreive buy id from sells record (buy_id)

In [ ]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = df_sells_latest['name']
name = names.to_string(index=False)
name

In [ ]:
sr_qty = df_sells_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = df_sells_latest['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = df_sells_latest['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = df_sells_latest['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

df_buys_latest = pd.read_sql(sql, conpf)
df_buys_latest.style.format(format_dict)

In [ ]:
transaction, name, qty, sell_price, buy_price

### Begin of Update buy table in MySQL stock database from sale transaction

In [ ]:
old_qty = df_buys_latest['qty'].iloc[0]
old_qty

In [ ]:
new_qty = old_qty + qty
print(new_qty)

In [ ]:
print(transaction, name, new_qty, buy_price)

In [ ]:
show_buy(const, name)

In [ ]:
update_buy(const, name, new_qty, buy_price)

In [ ]:
show_buy(const, name)

In [ ]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0 AND active = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [ ]:
show_dividend(const, name)

In [ ]:
new_qty = df_dividend['shares'].iloc[0] + qty
new_qty

In [ ]:
update_dividend(const, name, new_qty)

In [ ]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0 AND actual = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

In [ ]:
show_dividend(const, name)

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [ ]:
show_stocks(conlite, name)

In [ ]:
sql = """
    SELECT name, cost, available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)
print(sql)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks

In [ ]:
old_qty = int(df_stocks['available_qty'].iloc[0])
old_qty

In [ ]:
qty

In [ ]:
new_qty = old_qty + qty
new_qty

In [ ]:
new_buy_target = 24.10
new_sell_target = 24.30
new_buy_qty = 5000

In [ ]:
print(name, transaction, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

In [ ]:
update_stock(conlite, name, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

In [ ]:
show_stocks(conlite, name)

### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

In [ ]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' \
ORDER BY period, buy.name".format(yesterday)
print(sql)

In [ ]:
df_buy = pd.read_sql(sql, const)
df_buy

In [ ]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [ ]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

In [ ]:
df_buy.to_csv(output_file, header=True, index=False)
df_buy.to_csv(icd_file, header=True, index=False)
df_buy.to_csv(god_file, header=True, index=False)
df_buy.to_csv(osd_file, header=True, index=False)

In [ ]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

df_buy.to_excel(xsl_file, index=False)